# Jobbe: Help, me find a job!

Unfinished google capstone project.

Supposed to find job listings on company websites with simple user input.
Future work: fully automated pipeline


## Environment Setup

In [ ]:
import os
from google.genai import types
from google.adk.agents import LlmAgent, RunConfig
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
import asyncio
from google.adk.agents import SequentialAgent
from google.adk.models import Gemini

# taken from the google ai agent notebooks
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# change to different models if needed
# computer-use model is required in this version
llm_model_config = {
    "computer-use":"gemini-2.5-computer-use-preview-10-2025",
    "low-performance":"gemini-2.0-flash",
    "high-performance":"gemini-2.0-flash", # should be replaced by better model
}

print("✅ environment is set up")

## API Configuration

In [ ]:
# for local run, expects .env file with GOOGLE_API_KEY and GOOGLE_GENAI_USE_VERTEXAI=FALSE
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    # for kaggle notebook
    from kaggle_secrets import UserSecretsClient
    try:
        GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("✅ Setup and authentication complete.")
    except Exception as e:
        print(
            f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
        )
print("✅ api key loaded")

In [ ]:
# playwright setup
# taken from https://ai.google.dev/gemini-api/docs/computer-use#implement-computer-use
from playwright.sync_api import sync_playwright

# Browser configuration
BROWSER_HEADLESS = True
PAGE_LOAD_TIMEOUT = 30000  # 30 seconds in milliseconds
NAVIGATION_TIMEOUT = 5000   # 5 seconds for specific navigation


# 1. Configure screen dimensions for the target environment
SCREEN_WIDTH = 1440
SCREEN_HEIGHT = 900

# 2. Start the Playwright browser
# In production, utilize a sandboxed environment.
playwright = sync_playwright().start()
# Set headless=False to see the actions performed on your screen
browser = playwright.chromium.launch(headless=False)

# 3. Create a context and page with the specified dimensions
context = browser.new_context(
    viewport={"width": SCREEN_WIDTH, "height": SCREEN_HEIGHT}
)
page = context.new_page()

# 4. Navigate to an initial page to start the task
page.goto("https://www.google.com")



## Tool Functions

## Function declarations

## Memory System

## Agents

In [ ]:
# Navigator
navigator = LlmAgent(
    name="Navigator",
    model=Gemini(model=llm_model_config["browser-use"],
    retry_options=retry_config),
    description="",
    instruction="""

    """,
    sub_agents=[],
    tools=[]
)

In [ ]:
# Scraper
scraper = LlmAgent(
    name="Scraper",
    model=Gemini(model=llm_model_config["low_performance"],
    retry_options=retry_config),
    description="",
    instruction=
    """

    """,
    sub_agents=[],
    tools=[]
)

In [ ]:
# Filterer
filterer = LlmAgent(
name = "Filterer",
model = Gemini(model=llm_model_config["low_performance"],
               retry_options=retry_config),
description = "",
instruction =
"""

""",
sub_agents = [],
tools = []
)

In [ ]:
# Formatter
formatter = LlmAgent(
    name="Formatter",
    model=Gemini(model=llm_model_config["low_performance"],
    retry_options=retry_config),
    description="",
    instruction=
    """

    """,
    sub_agents=[],
    tools=[]
)

In [ ]:
# Coordinator
coordinator = LlmAgent(
    name="Coordinator",
    model=Gemini(model=llm_model_config["low_performance"],
    retry_options=retry_config),
    description="",
    instruction="""

    """,
    sub_agents=[],
    tools=[]
)

In [ ]:
# create the job search pipeline
root_agent = SequentialAgent(
    name="JobSearchPipeline",
    sub_agents=[coordinator, navigator, scraper, filterer,formatter],
)

runner = Runner(
    agent=root_agent,
    session_service=InMemorySessionService()
)

## Demo

In [ ]:
response = await runner.run_debug(
    user_id = "user_123",
    session_id = "session_abc",
    user_messages="look for AI Engineer jobs on https://jobs.towardsai.net/",
)